# Comparison of Replay Strategies in pyCLAD

This notebook compares the performance of `ReplayEnhancedStrategy` and `BalancedReservoirSamplingStrategy` from the `pyCLAD` library.
The comparison is performed across four datasets: Energy, NSL-KDD, UNSW, and Wind.
For each dataset, three different data scenarios are used: `random_anomalies`, `clustered_with_closest_assignment`, and `clustered_with_random_assignment`.

The notebook is divided into three main parts:
1.  **Setup**: Imports necessary libraries and defines the configuration for datasets, strategies, and output directories.
2.  **Run Experiments**: Executes the experiments for each combination of dataset, scenario, and strategy, saving the results to JSON files.
3.  **Analyze Results**: Loads the saved results, generates heatmaps for each experiment, and creates comparison heatmaps to visualize the performance difference between the two strategies.

In [1]:
import pathlib

# Datasets
from pyclad.data.datasets.unsw_dataset import UnswDataset
from pyclad.data.datasets.nsl_kdd_dataset import NslKddDataset
from pyclad.data.datasets.wind_energy_dataset import WindEnergyDataset
from pyclad.data.datasets.energy_plants_dataset import EnergyPlantsDataset

# Scenarios
from pyclad.scenarios.concept_aware import ConceptAwareScenario

from pyclad.metrics.continual.average_continual import ContinualAverage
from pyclad.metrics.continual.backward_transfer import BackwardTransfer
from pyclad.metrics.continual.forward_transfer import ForwardTransfer
# Models
from pyclad.models.adapters.pyod_adapters import LocalOutlierFactorAdapter

# Strategies
from pyclad.strategies.replay.replay import ReplayEnhancedStrategy
from pyclad.strategies.replay.candi import CandiStrategy

# Additional imports for replay strategies
from pyclad.strategies.replay.buffers.adaptive_balanced import (
    AdaptiveBalancedReplayBuffer,
)
from pyclad.strategies.replay.selection.random import RandomSelection

# Callback and metrics
from pyclad.callbacks.evaluation.concept_metric_evaluation import ConceptMetricCallback
from pyclad.callbacks.evaluation.memory_usage import MemoryUsageCallback
from pyclad.callbacks.evaluation.time_evaluation import TimeEvaluationCallback
from pyclad.metrics.base.roc_auc import RocAuc
from pyclad.output.json_writer import JsonOutputWriter

# Configuration
DATASETS = {
    "wind": WindEnergyDataset,
    "unsw": UnswDataset,
    "energy": EnergyPlantsDataset,
    "nsl-kdd": NslKddDataset,
}

DATASET_TYPES = [
    "random_anomalies",
    "clustered_with_closest_assignment",
    "clustered_with_random_assignment",
]

max_size = 2000

STRATEGIES = {
    "replay_enhanced": lambda model: ReplayEnhancedStrategy(
        model,
        AdaptiveBalancedReplayBuffer(selection_method=RandomSelection(), max_size=max_size),
    ),
    "watch_iqr": lambda model: CandiStrategy(
        model, max_buffer_size=max_size, threshold_ratio=0.5, warm_up_period=2, threshold_cal_index=0, resize_new_regime=True, boundary_aware=False
    ),
    "watch_percentile": lambda model: CandiStrategy(
        model, max_buffer_size=max_size, threshold_ratio=0.5, warm_up_period=2, threshold_cal_index=1, resize_new_regime=True, boundary_aware=False
    ),
    "watch_combined": lambda model: CandiStrategy(
        model, max_buffer_size=max_size, threshold_ratio=0.5, warm_up_period=2, threshold_cal_index=2, resize_new_regime=True, boundary_aware=False
    ),
}

RESULTS_DIR = pathlib.Path("comparison_results")
PLOTS_DIR = pathlib.Path("comparison_plots")
RESULTS_DIR.mkdir(exist_ok=True)
PLOTS_DIR.mkdir(exist_ok=True)

print("Setup complete.")

/Users/firepond/code/pyCLAD/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete.


### Run Experiments

The following cell runs the experiments. It iterates through each dataset and dataset type, and for each, it runs both the `ReplayEnhancedStrategy` and the `BalancedReservoirSamplingStrategy`. The results are saved in the `comparison_results` directory.

**Note:** This process can be time-consuming.

In [ ]:
def run_experiments():
    for dataset_name, dataset_class in DATASETS.items():
        for dataset_type in DATASET_TYPES:
            print(f"Running experiments for {dataset_name} - {dataset_type}")
            try:
                dataset = dataset_class(dataset_type=dataset_type)
            except Exception as e:
                print(f"Could not load dataset {dataset_name} with type {dataset_type}. Error: {e}")
                continue

            for strategy_name, strategy_builder in STRATEGIES.items():
                print(f"  with strategy: {strategy_name}")
                model = LocalOutlierFactorAdapter()
                strategy = strategy_builder(model)

                callbacks = [
                    ConceptMetricCallback(
                        base_metric=RocAuc(),
                        metrics=[ContinualAverage(), BackwardTransfer(), ForwardTransfer()]
                    ),
                    TimeEvaluationCallback(),
                    MemoryUsageCallback(),
                ]
                scenario = ConceptAwareScenario(dataset, strategy=strategy, callbacks=callbacks)
                scenario.run()

                # callbacks[0].print_continual_average()
                cur_continual_average = callbacks[0].get_continual_average()

            print("-" * 20)
            
        print(f"Finished experiments for {dataset_name}.")
        print("=" * 40)

run_experiments()
print("All experiments finished.")

Running experiments for wind - random_anomalies
  with strategy: replay_enhanced
Continual Average: 0.9722411774474681
  with strategy: watch_iqr
Continual Average: 0.9744344234816771
  with strategy: watch_percentile
Continual Average: 0.9741533996415428
  with strategy: watch_combined
Continual Average: 0.9730655830977748
--------------------
Running experiments for wind - clustered_with_closest_assignment
  with strategy: replay_enhanced
Continual Average: 0.9485569922713518
  with strategy: watch_iqr
Continual Average: 0.9519073633170784
  with strategy: watch_percentile
Continual Average: 0.9503649757138765
  with strategy: watch_combined
Continual Average: 0.9495187010383664
--------------------
Running experiments for wind - clustered_with_random_assignment
  with strategy: replay_enhanced
Continual Average: 0.963090051093965
  with strategy: watch_iqr
Continual Average: 0.9699789717511103
  with strategy: watch_percentile
Continual Average: 0.9706419547180922
  with strategy: w